## fine-tune tensorflow distilBert and save model to cloud for usage

In [3]:
import sys
!{sys.executable} -m pip install -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
detectron2 0.5+cu110 requires black==21.4b2, but you have black 21.9b0 which is incompatible.
prefect 0.15.6 requires requests<2.26,>=2.20, but you have requests 2.26.0 which is incompatible.
libcst 0.3.21 requires pyyaml>=5.2, but you have pyyaml 5.1 which is incompatible.
bokeh 2.4.0 requires typing_extensions>=3.10.0, but you have typing-extensions 3.7.4.3 which is incompatible.
black 21.9b0 requires typing-extensions>=3.10.0.0, but you have typing-extensions 3.7.4.3 which is incompatible.
awscli 1.19.106 requires s3transfer<0.5.0,>=0.4.0, but you have s3transfer 0.5.0 which is incompatible.


In [4]:
import warnings
pd.options.display.max_rows = 999
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import pickle

import logging
tf.get_logger().setLevel(logging.ERROR)

2021-10-09 21:42:05.095224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-09 21:42:05.095275: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
x = [
     'Great customer service! The food was delicious! Definitely a come again.',
     'The VEGAN options are super fire!!! And the plates come in big portions. Very pleased with this spot, I\'ll definitely be ordering again',
     'Come on, this place is family owned and operated, they are super friendly, the tacos are bomb.',
     'This is such a great restaurant. Multiple times during days that we don\'t want to cook, we\'ve done takeout here and it\'s been amazing. It\'s fast and delicious.',
     'Staff is really nice. Food is way better than average. Good cost benefit.',
     'pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top.',
     'At such a *fine* institution, I find the lack of knowledge and respect for the art appalling',
     'If I could give one star I would...I walked out before my food arrived the customer service was horrible!',
     'Wow the slowest drive thru I\'ve ever been at WOWWWW. Horrible I won\'t be coming back here ever again',
     'Service: 1 out of 5 stars. They will mess up your order, not have it ready after 30 mins calling them before. Worst ran family business Ive ever seen.'
]

y = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [4]:
MODEL_NAME = 'distilbert-base-uncased'
MAX_LEN = 20

review = x[0]

tkzr = DistilBertTokenizer.from_pretrained(MODEL_NAME)

inputs = tkzr(review, max_length=MAX_LEN, truncation=True, padding=True)

print(f'review: \'{review}\'')
print(f'input ids: {inputs["input_ids"]}')
print(f'attention mask: {inputs["attention_mask"]}')

review: 'Great customer service! The food was delicious! Definitely a come again.'
input ids: [101, 2307, 8013, 2326, 999, 1996, 2833, 2001, 12090, 999, 5791, 1037, 2272, 2153, 1012, 102]
attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [5]:
def construct_encodings(x, tkzr, max_len, trucation=True, padding=True):
    return tkzr(x, max_length=max_len, truncation=trucation, padding=padding)
    
encodings = construct_encodings(x, tkzr, max_len=MAX_LEN)

In [6]:
def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))
    
tfdataset = construct_tfdataset(encodings, y)


2021-09-12 22:10:33.243659: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-12 22:10:33.244027: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-09-12 22:10:33.244040: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-12 22:10:33.244062: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-stella-40dev2vec-2ecom): /proc/driver/nvidia/version does not exist
2021-09-12 22:10:33.244301: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operat

In [7]:
TEST_SPLIT = 0.2
BATCH_SIZE = 2

train_size = int(len(x) * (1-TEST_SPLIT))

tfdataset = tfdataset.shuffle(len(x))
tfdataset_train = tfdataset.take(train_size)
tfdataset_test = tfdataset.skip(train_size)

tfdataset_train = tfdataset_train.batch(BATCH_SIZE)
tfdataset_test = tfdataset_test.batch(BATCH_SIZE)

## fine tune the model

In [8]:
N_EPOCHS = 10

model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
optimizer = optimizers.Adam(learning_rate=3e-5)
loss = losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.fit(tfdataset_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS)

2021-09-12 22:10:36.006811: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint 

Epoch 1/10


2021-09-12 22:10:43.159486: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-12 22:10:43.211938: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2999995000 Hz


4/4 [==============================] - 9s 289ms/step - loss: 0.6869 - accuracy: 0.5500
Epoch 2/10
4/4 [==============================] - 1s 292ms/step - loss: 0.6949 - accuracy: 0.4833
Epoch 3/10
4/4 [==============================] - 1s 284ms/step - loss: 0.6263 - accuracy: 0.7667
Epoch 4/10
4/4 [==============================] - 1s 289ms/step - loss: 0.5616 - accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 1s 278ms/step - loss: 0.4782 - accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 1s 283ms/step - loss: 0.3332 - accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 1s 281ms/step - loss: 0.2443 - accuracy: 1.0000
Epoch 8/10
4/4 [==============================] - 1s 278ms/step - loss: 0.1890 - accuracy: 1.0000
Epoch 9/10
4/4 [==============================] - 1s 279ms/step - loss: 0.1391 - accuracy: 1.0000
Epoch 10/10
4/4 [==============================] - 1s 278ms/step - loss: 0.1004 - accuracy: 1.0000


## test the model

In [9]:
benchmarks = model.evaluate(tfdataset_test, return_dict=True, batch_size=BATCH_SIZE)
print(benchmarks)

1/1 [==============================] - 1s 1s/step - loss: 0.0707 - accuracy: 1.0000
{'loss': 0.07065244019031525, 'accuracy': 1.0}


In [10]:
def create_predictor(model, model_name, max_len):
    tkzr = DistilBertTokenizer.from_pretrained(model_name)
    def predict_proba(text):
        x = [text]
        encodings = construct_encodings(x, tkzr, max_len=max_len)
        tfdataset = construct_tfdataset(encodings)
        tfdataset = tfdataset.batch(1)

        preds = model.predict(tfdataset).logits
        preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()
        return preds[0][0]
    
    return predict_proba

clf = create_predictor(model, MODEL_NAME, MAX_LEN)
print(clf('this restaurant has horrible food'))

0.13448104


## (Optional) save model to cloud for checkpointing

In [11]:
## save model locally
model.save_pretrained('/root/model/clf')
with open('/root/model/info.pkl', 'wb') as f:
    pickle.dump((MODEL_NAME, MAX_LEN), f)

In [12]:
def upload_to_cloud(local_file_name, remote_file_name):
    import os
    import s3fs
    import gcsfs
    cloud_name = remote_file_name.split('://')[0]
    if cloud_name =='gs':
        fs = gcsfs.GCSFileSystem(project=os.environ['GCP_PROJECT'])
    elif cloud_name =='s3':
        fs = s3fs.S3FileSystem()
    else:
        raise NameError(f'cloud name {cloud_name} unknown')
    try:    
        print(f'upload {local_file_name} to {remote_file_name}')
        fs.put(local_file_name, remote_file_name, recursive=True)
        print("done uploading!")
    except Exception as exp:
        print(f"upload failed: {exp}")
        
    return 

**Note: the remote_file_path is a path of a GCP or S3 bucket that connected to your Hyperplane cluster**

In [50]:

model_file_path = 'model'
remote_file_path = f"s3://d2v-tmp/demo/bach_inference/{model_file_path}"

upload_to_cloud(model_file_path, remote_file_path)


upload model to gs://pipeline_data/demo/bach_inference/model
done uploading!
